In [68]:
import numpy as np
import pandas as pd

import os
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import norm
from scipy import stats


from sklearn_pandas import DataFrameMapper
from sklearn_pandas import CategoricalImputer
#from sklearn.preprocessing import Imputer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer

from sklearn.base import TransformerMixin
from sklearn.compose import ColumnTransformer


from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

%matplotlib inline

In [69]:
class Categorical_Imputer(TransformerMixin):
    def __init__(self):
        """Impute missing values.

        Columns of dtype object are imputed with the most frequent value 
        in column.

        Columns of other types are imputed with mean of column.

        """
        #self.strategy = strategy


    def fit_transform(self,X,y=None):
        col_all = X.columns
        for col in col_all:
            #print(col,' ',X[col].value_counts().index[0])
            X[col].fillna(X[col].value_counts().index[0],inplace=True)
            #X.info()
        return X
 

In [70]:
train_df = pd.read_csv('train.csv')
test_df  = pd.read_csv('test.csv')

y_train = train_df['SalePrice']
train_df = train_df.drop(['Id','SalePrice'],axis=1)
test_df = test_df.drop('Id',axis=1)



test_length = len(test_df)
train_length = len(train_df)

In [71]:
all_sample = pd.concat([train_df,test_df])
all_sample['MSSubClass'] = pd.Categorical(all_sample.MSSubClass)
num_col = list(all_sample._get_numeric_data().columns)
cat_col = list(set(all_sample.columns) - set(num_col))


In [72]:
# select numeric
select_numeric = FunctionTransformer(lambda x : x[num_col],validate=False)

In [73]:
select_cate = FunctionTransformer(lambda x : x[cat_col],validate=False)

In [74]:
cate_imputer = Categorical_Imputer()
cate_imp = FunctionTransformer(cate_imputer.fit_transform,validate=False)

In [75]:
get_dummies = FunctionTransformer(lambda x: pd.get_dummies(x, prefix_sep='_', drop_first=True), 
                                  validate=False)

Create a pipeline to handle the numeric data
steps
- Select
- imputer (simple mode Imputer is used now. A class can be used in the future)
- feature enginering (optional in the future)

In [76]:
#simple mode for the imputer. A class can be written in the future
num_imputer = SimpleImputer(strategy="most_frequent")
num_standard = StandardScaler()


num_pipeline = Pipeline(steps=
                        [('select_num',select_numeric),
                         ('imp_num',num_imputer),
                        ('standadize',num_standard)])
num_pipeline.fit_transform(all_sample)

array([[-0.12767816, -0.21787869,  0.6461834 , ..., -0.08959196,
        -1.55218353,  0.15764553],
       [ 0.56712524, -0.07204408, -0.06318454, ..., -0.08959196,
        -0.44692486, -0.60296158],
       [ 0.01128252,  0.13719688,  0.6461834 , ..., -0.08959196,
         1.02675338,  0.15764553],
       ...,
       [ 4.27274338,  1.24680804, -0.77255247, ..., -0.08959196,
         1.02675338, -1.36356868],
       [-0.26663884,  0.0346054 , -0.77255247, ...,  1.14431218,
         0.28991426, -1.36356868],
       [ 0.28920388, -0.06862014,  0.6461834 , ..., -0.08959196,
         1.7635925 , -1.36356868]])

create a pipeline to handle categorical data
steps
    - select
    - imputer
    - get_dummy

In [77]:
cate_pipeline = Pipeline(steps=[
                                ('select_cate',select_cate),
                                ('imp_cate',cate_imp),
                                ('get_dummy',get_dummies)
])
cate_pipeline.fit_transform(all_sample)

/Users/kehindetomiwa/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3660: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


,HeatingQC_Fa,HeatingQC_Gd,HeatingQC_Po,HeatingQC_TA,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CmentBd,Exterior2nd_HdBoard,...,GarageType_CarPort,GarageType_Detchd,BsmtCond_Gd,BsmtCond_Po,BsmtCond_TA,Street_Pave,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
3,0,1,0,0,0,0,0,0,0,0,...,0,1,1,0,0,1,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1
7,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,1,0,0,0,1
8,0,1,0,0,0,0,0,0,0,0,...,0,1,0,0,1,1,1,0,0,0
9,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,1,0,0,0,1


In [78]:
#add cate and numeric
union = ColumnTransformer([
    ('numerical',num_pipeline,num_col),
    ('categorical',cata_pipeline,cat_col)
])
union_result = union.fit_transform(all_sample)

In [79]:
union_result.shape

(2919, 259)

In [80]:
X_train = union_result[:train_length]
X_test  = union_result[train_length:]

In [81]:

from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

#xgb_Reg = xgb.XGBRegressor()



#Dtree_reg = DecisionTreeRegressor(random_state=42)

#ridgeReg = Ridge(alpha=10,normalize=True)
#lasso_Reg = Lasso(alpha=2,normalize=True)
#EN_Reg = ElasticNet(alpha=5,l1_ratio=0.5,normalize=False)


models = {
    'RidgeRegression':Ridge(),
    'LassoRegression':Lasso(),
    'ElasticNetRegression':ElasticNet(),
    'XBoostRegression':XGBRegressor()
}


params = {
    'RidgeRegression':{
        "alpha":[1,5,10,20],
        "normalize":[True,False]
    },
    'LassoRegression':{
        "alpha":[1,5,10,20],
        "normalize":[True,False]
    },
    'ElasticNetRegression':{
        "alpha":[1,5,10,20],
        "normalize":[True,False],
        "l1_ratio":[0.5]
    },
    'XBoostRegression':{
        "learning_rate":[0.05,0.1],
        "n_estimators":[10,50,100]
    }
    
    
}

In [ ]:
from sklearn.model_selection import GridSearchCV

def model_fit(train_features, train_actuals):
        """
        fits the list of models to the training data, thereby obtaining in each 
        case an evaluation score after GridSearchCV cross-validation
        """
        for name in models.keys():
            est = models[name]
            est_params = params[name]
            gscv = GridSearchCV(estimator=est, param_grid=est_params, cv=5,
                               scoring='neg_mean_squared_error', return_train_score=True)
            gscv.fit(train_actuals, train_features)
            cvres = gscv.cv_results_
            print("best parameters are: {}".format(gscv.best_estimator_))
            for mean_score,par in zip(cvres["mean_test_score"],cvres["params"]):
                print(np.sqrt(-mean_score), par)
            
            
            
model_fit(y_train,X_train)
